In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import keras
keras.__version__

'2.15.0'

In [138]:
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image

# La imagen que adoptará el estilo
target_image_path = "gato.jpg"

img = Image.open(target_image_path)
img_width, img_height = img.size

# La imagen que dará el estilo
style_reference_image_path = "Estilo.jpg"

# Dimensiones de la imagen generada
width, height = load_img(target_image_path).size
img_height = img_height
img_width = img_width

In [139]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]  # BGR -> RGB
    x = np.clip(x, 0, 225).astype("float64")
    return x

In [140]:
from keras import backend as k

target_image = k.constant(preprocess_image(target_image_path))
style_reference_image = k.constant(preprocess_image(style_reference_image_path))

# Este marcador de posición contendrá nuestra imagen generada
combination_image = k.placeholder((1, img_height, img_width, 3))

# Combinamos las 3 imagenes en un solo lote
input_tensor = k.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor,
                    weights= "imagenet",
                    include_top=False)
print("Modelo cargado correctamente.")

Modelo cargado correctamente.


In [141]:
def content_loss(base, combination):
    return k.sum(k.square(combination - base))

In [142]:
def gram_matrix(x):
    features = k.batch_flatten(k.permute_dimensions(x, (2,0,1)))
    gram = k.dot(features, k.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return k.sum(k.square(S-C)) / (4. * (channels ** 2) * (size ** 2))

In [143]:
def total_variation_loss(x):
    a = k.square(
        x[:, :img_height -1, :img_width -1, :] - x[:, 1:, :img_width -1, :])
    b = k.square(
        x[:, :img_height -1, :img_width -1, :] - x[:, :img_height -1, 1:, :])
    return k.sum(k.pow(a+b, 1.25))

In [144]:
# Crear un diccionario que asocia nombres de capas con sus salidas
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Nombre de la capa usada para la pérdida de contenido
content_layer = "block5_conv2"

# Nombre de las capas usadas para la pérdida de estilo
style_layers = ["block1_conv1",
                "block2_conv1",
                "block3_conv1",
                "block4_conv1",
                "block5_conv1"]

# Pesos para las diferentes pérdidas en el cálculo total
total_variation_weight = 1e-4  # Peso para la pérdida de variación total
style_weight = 1.0             # Peso para la pérdida de estilo
content_weight = 0.025         # Peso para la pérdida de contenido

# Inicializar la variable de pérdida total
loss = k.variable(0.0)

# Extraer características de la capa de contenido
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]   # Características de la imagen objetivo
combination_features = layer_features[2, :, :, :]    # Características de la imagen combinada
loss = loss + content_weight * content_loss(target_image_features, combination_features)  # Añadir la pérdida de contenido a la pérdida total

# Calcular y añadir la pérdida de estilo para cada capa de estilo
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]  # Características de la imagen de referencia de estilo
    combination_features = layer_features[2, :, :, :]      # Características de la imagen combinada
    sl = style_loss(style_reference_features, combination_features)  # Calcular la pérdida de estilo
    loss += (style_weight / len(style_layers)) * sl        # Añadir la pérdida de estilo ponderada a la pérdida total

# Añadir la pérdida de variación total a la pérdida total
loss += total_variation_weight * total_variation_loss(combination_image)


In [145]:
import tensorflow as tf

# Función para calcular la pérdida y los gradientes
@tf.function
def compute_loss_and_grads(image):
    with tf.GradientTape() as tape:
        tape.watch(image)
        loss = total_variation_weight * total_variation_loss(image)
    grads = tape.gradient(loss, image)
    return loss, grads

# Clase Evaluator para evaluar la pérdida y los gradientes
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    # Método para calcular la pérdida
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        x_tensor = tf.convert_to_tensor(x)
        loss_value, grad_values = compute_loss_and_grads(x_tensor)
        self.loss_value = loss_value.numpy()
        self.grad_values = grad_values.numpy().flatten().astype("float64")
        return self.loss_value

    # Método para obtener los gradientes
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# Dimensiones de la imagen
img_height = 400
img_width = 1846

# Instancia de la clase Evaluator
evaluator = Evaluator()

In [150]:
from scipy.optimize import fmin_l_bfgs_b
import time
import imageio

# Prefijo para los nombres de los archivos de resultado
result_prefix = "style_transfer_result"
# Número de iteraciones de optimización
iterations = 1

# Preprocesar la imagen de destino
x = preprocess_image(target_image_path)
# Aplanar la imagen preprocesada para la optimización
x = x.flatten()
# Bucle para realizar las iteraciones de optimización
for i in range(iterations):
    print("Inicio de iteraciones", i)
    start_time = time.time()  # Registrar el tiempo de inicio
    # Ejecutar la optimización L-BFGS-B
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                      fprime=evaluator.grads, maxfun=20)
    print("Valos de perdida actual:", min_val)

    # Copiar y remodelar la imagen optimizada
    img = x.copy().reshape((img_height, img_width, 3))
    # Deshacer el preprocesamiento para obtener la imagen final
    img = deprocess_image(img)
    # Generar el nombre del archivo de resultado
    fname = result_prefix + "_at_iteration_%d.png" % i
    
    # Guardar la imagen resultante
    imageio.imwrite('Imagen generada.png', img)


    end_time = time.time()  # Registrar el tiempo de finalización
    print("Iteración %d completa en %ds" % (i, end_time - start_time))


Inicio de iteraciones 0
Valos de perdida actual: 16798.581819593477
Iteración 0 completa en 11s


In [149]:
img

array([[[170.33752249, 136.24846544, 101.51468585],
        [167.91257113, 133.7212046 ,  99.30661454],
        [164.06197985, 129.80403166,  95.61315769],
        ...,
        [218.16342017, 174.16342115, 135.70574809],
        [217.99516204, 173.99516301, 135.40440374],
        [217.7294119 , 173.72941288, 134.99855021]],

       [[170.32336046, 136.24251946, 101.56017764],
        [168.15117192, 133.97714824,  99.63348677],
        [164.46181985, 130.22858468,  96.114531  ],
        ...,
        [217.93626577, 173.93626675, 135.8175069 ],
        [218.18300874, 174.18300972, 135.8883581 ],
        [218.42628329, 174.42628427, 136.02067551]],

       [[170.70014416, 136.65287001, 102.25103579],
        [168.78468226, 134.65665258, 100.66778056],
        [165.14574537, 130.97508817,  97.207322  ],
        ...,
        [218.1532539 , 174.15325488, 136.18010134],
        [217.96257855, 173.96257952, 136.15908833],
        [217.78449903, 173.78450001, 136.20669824]],

       ...,

      